# CUDA を使って画像に3DLUTをかける

## 3DLUT の補間処理の元ネタ

FUJIFILM社の「LOGとLUTの基礎講座」資料 P.38 のアルゴリズムを使用する。

![figure1](./figure/3dlut_interpolation.jpg)

リンク：[LOGとLUTの基礎講座](http://fujifilm.jp/business/broadcastcinema/solution/color_management/is-mini/promotion/pack/pdf/news/20150122_jppa.pdf)

## Python向けに補間処理の数式を考える。

![figure1](./figure/sekkei.jpg)

8頂点で囲われた立方体を上図を参考に8領域に分ける。それぞれの領域の体積を$V_0 \sim V_7$ と置くと、以下の式で算出できる。

\begin{align}
V_0 & = R_0 \cdot G_0 \cdot B_0 \\
V_1 & = R_0 \cdot G_0 \cdot B_1 \\
V_2 & = R_0 \cdot G_1 \cdot B_0 \\
V_3 & = R_0 \cdot G_1 \cdot B_1 \\
V_4 & = R_1 \cdot G_0 \cdot B_0 \\
V_5 & = R_1 \cdot G_0 \cdot B_1 \\
V_6 & = R_1 \cdot G_1 \cdot B_0 \\
V_7 & = R_1 \cdot G_1 \cdot B_1 \\
\end{align}

$$
\begin{align}
R_{out} & = V_0 \cdot L_7[R] + V_1 \cdot L_6[R] + V_2 \cdot L_5[R] + V_3 \cdot L_4[R] \\
        & + V_4 \cdot L_3[R] + V_5 \cdot L_2[R] + V_6 \cdot L_1[R] + V_7 \cdot L_0[R] \\
G_{out} & = V_0 \cdot L_7[G] + V_1 \cdot L_6[G] + V_2 \cdot L_5[G] + V_3 \cdot L_4[G] \\ 
        & + V_4 \cdot L_3[G] + V_5 \cdot L_2[G] + V_6 \cdot L_1[G] + V_7 \cdot L_0[G] \\
B_{out} & = V_0 \cdot L_7[B] + V_1 \cdot L_6[B] + V_2 \cdot L_5[B] + V_3 \cdot L_4[B] \\
        & + V_4 \cdot L_3[B] + V_5 \cdot L_2[B] + V_6 \cdot L_1[B] + V_7 \cdot L_0[B]
\end{align}
$$

## Grid数を考慮した数式を立ててみよう

グリッド数を$N$とする。$N$ は整数 $m$ によって以下の式で求められる。<br>
ひとまずは、m=4, 5, 6 あたりをターゲットとする。

$$
N = 2^m + 1
$$

入力画像を $D(w, h, c)$ とする。$w, h$ は空間方向の座標を $c$ は 色方向の座標を意味する。<br>
入力画像は$(N - 1)$に合わせて正規化する。正規化後の入力画像を $D'$ とする。

$$
D' = \frac{D}{{\rm max}(D)} \cdot (N - 1)
$$

さて、この時 $D'$ が参照するグリッドのインデックス $i, j, k$ は以下となる。<br>
※ $D=N$ つまり最大値の時に、本当に端っこのグリッドに当たるか不安。要確認。

$$
i = \lfloor D' \rfloor
$$

# 3DLUT の 読み込みと書き込み

## 使う形式
**.cube** 形式を使う。詳細は Adobe の [Cube LUT Specification](http://wwwimages.adobe.com/content/dam/Adobe/en/products/speedgrade/cc/pdfs/cube-lut-specification-1.0.pdf) 
参照。

## ザックリと .cube 形式について説明

### データの中身
エディタで .cube を開くと以下のようになってる。<br>
したの方に実際の 3DLUT データが詰まってる。<br>
データは左から順に Red/Green/Blue となっている。
```
# DaVinci Resolve Cube (3D LUT).
#
# Converts Rec.709 data to DCI-XYZ.
# Gamma as defined by ITU-Rec.1886.
# 

LUT_3D_SIZE 33
LUT_3D_INPUT_RANGE 0.0000000000 1.0000000000

0.0000000000 0.0000000000 0.0000000000
0.0280636951 0.0217522469 0.0086490514
0.0532131180 0.0412456335 0.0163999427
0.0773685559 0.0599685796 0.0238444942
0.1009003238 0.0782081173 0.0310968346
0.1239789517 0.0960964251 0.0382095201
0.1467027800 0.1137097266 0.0452128587
0.1691357380 0.1310975738 0.0521265529
0.1913226612 0.1482947187 0.0589644207
0.2132966958 0.1653268532 0.0657366777
0.2350833187 0.1822137243 0.0724511756
0.2567027062 0.1989709707 0.0791141326
0.2781712211 0.2156112755 0.0857305916
0.2995023928 0.2321451250 0.0923047223
(以下省略)
```

### 3DLUT のグリッドのインクリメントの順番
1行ごとに3DLUTの Index がインクリメントするが、その順序は Red, Green, Blue となっている。<br>
例えば、3x3x3 の 3DLUT だった場合、以下のように各色の Index が増加する。

| 行番号 | Red | Green | Blue | 
|-----|-----|-------|------| 
| 0   | 0   | 0     | 0    | 
| 1   | 1   | 0     | 0    | 
| 2   | 2   | 0     | 0    | 
| 3   | 0   | 1     | 0    | 
| 4   | 1   | 1     | 0    | 
| 5   | 2   | 1     | 0    | 
| 6   | 0   | 2     | 0    | 
| 7   | 1   | 2     | 0    | 
| 8   | 2   | 2     | 0    | 
| 9   | 0   | 0     | 1    | 
| 10  | 1   | 0     | 1    | 
| 11  | 2   | 0     | 1    | 
| 12  | 0   | 1     | 1    | 
| 13  | 1   | 1     | 1    | 
| 14  | 2   | 1     | 1    | 
| 15  | 0   | 2     | 1    | 
| 16  | 1   | 2     | 1    | 
| 17  | 2   | 2     | 1    | 
| 18  | 0   | 0     | 2    | 
| 19  | 1   | 0     | 2    | 
| 20  | 2   | 0     | 2    | 
| 21  | 0   | 1     | 2    | 
| 22  | 1   | 1     | 2    | 
| 23  | 2   | 1     | 2    | 
| 24  | 0   | 2     | 2    | 
| 25  | 1   | 2     | 2    | 
| 26  | 2   | 2     | 2    | 


In [40]:
# CUBE形式の3DLUTの読み書きモジュールを作る
# -----------------------------------------
import sys
import imp
sys.path.append('./code')
import pycuda_3dlut as p3
imp.reload(p3)
lut_data = p3.load_3dlut_cube('./data/Rec709 to DCI-XYZ.cube')
p3.save_3dlut_cube(lut_data, './data/save_test.cube')

lut_size = 33x33x33
data start line = 10


# 実際に3DLUTのデータを作る

3DLUTのデータ作成は簡単だ。IN-OUTの関係を明確にしてOUTをLUT化すれば良い。<br>
それでは実際にデータを作ってみよう。

## 入力データの作成
以下のコードで作る。


In [7]:
import numpy as np
grid_num = 3
x_r = (np.arange(grid_num**3) // (grid_num**0)) % grid_num
x_g = (np.arange(grid_num**3) // (grid_num**1)) % grid_num
x_b = (np.arange(grid_num**3) // (grid_num**2)) % grid_num

x = (np.dstack((x_r, x_g, x_b)) / (grid_num - 1)).astype(np.float32)

print(x)

[[[ 0.   0.   0. ]
  [ 0.5  0.   0. ]
  [ 1.   0.   0. ]
  [ 0.   0.5  0. ]
  [ 0.5  0.5  0. ]
  [ 1.   0.5  0. ]
  [ 0.   1.   0. ]
  [ 0.5  1.   0. ]
  [ 1.   1.   0. ]
  [ 0.   0.   0.5]
  [ 0.5  0.   0.5]
  [ 1.   0.   0.5]
  [ 0.   0.5  0.5]
  [ 0.5  0.5  0.5]
  [ 1.   0.5  0.5]
  [ 0.   1.   0.5]
  [ 0.5  1.   0.5]
  [ 1.   1.   0.5]
  [ 0.   0.   1. ]
  [ 0.5  0.   1. ]
  [ 1.   0.   1. ]
  [ 0.   0.5  1. ]
  [ 0.5  0.5  1. ]
  [ 1.   0.5  1. ]
  [ 0.   1.   1. ]
  [ 0.5  1.   1. ]
  [ 1.   1.   1. ]]]


## OUTデータ算出関数の作成

OUTデータを作るための関数を作る。関数の引数は `(in_data, **kwargs)` で統一したい。

In [2]:
import numpy as np


def rgb2yuv_for_3dlut(in_data, **kwargs):
    """
    # 概要
    RGB2YUVの3DLUTデータを作る。
    
    # 引数
    kwargs['mtx'] に行列の係数を入れておくこと。
    以下は例。
    
    ```
    matrix_param = np.array([[0.2126, 0.7152, 0.0722],
                            [-0.114572, -0.385428, 0.5],
                            [0.5, -0.454153, -0.045847]])
    kwargs = {'mtx' : matrix_param}
    ```

    """
    mtx = kwargs['mtx']
    r_in, g_in, b_in = np.dsplit(in_data, 3)
    y = r_in * mtx[0][0] + g_in * mtx[0][1] + b_in * mtx[0][2]
    u = r_in * mtx[1][0] + g_in * mtx[1][1] + b_in * mtx[1][2] + 0.5
    v = r_in * mtx[2][0] + g_in * mtx[2][1] + b_in * mtx[2][2] + 0.5
    
    out_data = np.dstack((v, y, u))  # 並びが V, Y, U であることに注意
    
    return out_data

# 入力データ作成
# -----------------
grid_num = 33
x_r = (np.arange(grid_num**3) // (grid_num**0)) % grid_num
x_g = (np.arange(grid_num**3) // (grid_num**1)) % grid_num
x_b = (np.arange(grid_num**3) // (grid_num**2)) % grid_num

x = (np.dstack((x_r, x_g, x_b)) / (grid_num - 1)).astype(np.float32)

# LUTデータ作成
# -----------------
matrix_param = np.array([[0.2126, 0.7152, 0.0722],
                         [-0.114572, -0.385428, 0.5],
                         [0.5, -0.454153, -0.045847]])
kwargs = {'mtx' : matrix_param}
lut = rgb2yuv_for_3dlut(in_data=x, **kwargs)

print(lut)

[[[ 0.5         0.          0.5       ]
  [ 0.515625    0.00664375  0.49641964]
  [ 0.53125     0.0132875   0.49283925]
  ..., 
  [ 0.46875     0.98671252  0.50716072]
  [ 0.484375    0.99335629  0.50358033]
  [ 0.5         1.          0.5       ]]]


## OUTデータ算出の枠組み作成

実際にOUTデータを作る際に、毎回入力データを定義するのは面倒なので、以下のように関数化をする。

In [3]:
def make_3dlut_data(grid_num=17, func=None, **kwargs):
    """
    # 概要
    3DLUTデータを作成する。

    # 詳細
    * func : 実際に処理をする関数を渡す
    * kwargs : func に引数として渡すやつ
    
    # 注意事項
    例によってエラー処理は皆無。トリッキーな呼び方はしないでね。
    """
    # 入力データ作成
    # -----------------
    x_r = (np.arange(grid_num**3) // (grid_num**0)) % grid_num
    x_g = (np.arange(grid_num**3) // (grid_num**1)) % grid_num
    x_b = (np.arange(grid_num**3) // (grid_num**2)) % grid_num
    
    x = (np.dstack((x_r, x_g, x_b)) / (grid_num - 1)).astype(np.float32)
    
    # LUTデータ作成
    # -----------------
    lut = func(in_data=x, **kwargs)
    
    # LUTデータは画像データと区別して、1次元配列っぽくする
    # ----------------------------------------------------
    if len(lut.shape) == 3:
        lut = lut.reshape((lut.shape[1], lut.shape[2]))
    
    return lut

# 実際に make_3dlut_data をコールしてみる
# ----------------------------------------
matrix_param = np.array([[0.2126, 0.7152, 0.0722],
                         [-0.114572, -0.385428, 0.5],
                         [0.5, -0.454153, -0.045847]])
kwargs = {'mtx' : matrix_param}

lut = make_3dlut_data(grid_num=17, func=rgb2yuv_for_3dlut, **kwargs)

print(lut)

[[ 0.5         0.          0.5       ]
 [ 0.53125     0.0132875   0.49283925]
 [ 0.5625      0.026575    0.48567849]
 ..., 
 [ 0.4375      0.97342497  0.51432145]
 [ 0.46875     0.98671252  0.50716072]
 [ 0.5         1.          0.5       ]]


## 3DLUT処理(x86版)

x86上で動作する Pythonコードの 3DLUT処理関数を仕上げよう。<br>
仕様は上の方に書いてあるので、それを実装するだけ。

In [21]:
import cv2
import numba
import sys


def img_open_and_normalize(filename):
    img = cv2.imread(filename, cv2.IMREAD_ANYDEPTH | cv2.IMREAD_COLOR)
    if img is None:
        print("ERROR!\n{} is not found".format(filename))
        sys.exit(1)
    try:
        img_max_value = np.iinfo(img.dtype).max
    except:
        img_max_value = 1.0
    img = np.float32(img/img_max_value)
    return img


@numba.jit
def exec_3dlut_on_x86(img, lut):
    """
    # 概要
    3DLUTを画像に適用する。

    # 詳細
    lut には Adobe CUBE 形式の順序で並んだ 3DLUTデータを
    Numpy の np.float32 で入れておくこと。
    
    # 備考
    本関数は将来的にCUDAに移植することを考えて
    forループを使って実装している。よってクソ遅い。
    耐えられないようであれば numba.jit のデコレータを使うこと。
    """

    # grid_num を算出しつつ lut の shape を確認
    # ------------------------------------------
    grid_num = np.uint8(np.round(np.power(lut.shape[0], 1/3)))
    total_index = grid_num**3
    if lut.shape[0] != total_index:
        print("error! 3DLUT size is invalid.")
        return None

    img = img * (grid_num - 1)
    out_img = np.empty_like(img)
    
    for w_idx in range(img.shape[1]):
        for h_idx in range(img.shape[0]):
            r = img[h_idx][w_idx][0]
            g = img[h_idx][w_idx][1]
            b = img[h_idx][w_idx][2]
            
            # r, g, b の各 Index を求める
            # ---------------------------
            r_idx = np.uint32(np.floor(r))
            g_idx = np.uint32(np.floor(g))
            b_idx = np.uint32(np.floor(b))
            idx_array = [r_idx, g_idx, b_idx]
            
            # 体積算出のために r_0 ～ b_1 を求める
            # ------------------------------------
            r_0 = r - r_idx
            r_1 = 1 - r_0
            g_0 = g - g_idx
            g_1 = 1 - g_0
            b_0 = b - b_idx
            b_1 = 1 - b_0
            
            # 体積計算
            # -------------------------------------
            v_0 = (r_0 * g_0 * b_0)
            v_1 = (r_0 * g_0 * b_1)
            v_2 = (r_0 * g_1 * b_0)
            v_3 = (r_0 * g_1 * b_1)
            v_4 = (r_1 * g_0 * b_0)
            v_5 = (r_1 * g_0 * b_1)
            v_6 = (r_1 * g_1 * b_0)
            v_7 = (r_1 * g_1 * b_1)
            # v_array = np.array([v_0, v_1, v_2, v_3, v_4, v_5, v_6, v_7])
            
            # l_0 ～ l_7 を事前に求めておく
            # --------------------------------------
            r_coef = grid_num ** 0
            g_coef = grid_num ** 1
            b_coef = grid_num ** 2

            l0_idx = (r_idx + 0)*(r_coef) + (g_idx + 0)*(g_coef) + (b_idx + 0)*(b_coef)
            l1_idx = (r_idx + 0)*(r_coef) + (g_idx + 0)*(g_coef) + (b_idx + 1)*(b_coef)
            l2_idx = (r_idx + 0)*(r_coef) + (g_idx + 1)*(g_coef) + (b_idx + 0)*(b_coef)
            l3_idx = (r_idx + 0)*(r_coef) + (g_idx + 1)*(g_coef) + (b_idx + 1)*(b_coef)
            l4_idx = (r_idx + 1)*(r_coef) + (g_idx + 0)*(g_coef) + (b_idx + 0)*(b_coef)
            l5_idx = (r_idx + 1)*(r_coef) + (g_idx + 0)*(g_coef) + (b_idx + 1)*(b_coef)
            l6_idx = (r_idx + 1)*(r_coef) + (g_idx + 1)*(g_coef) + (b_idx + 0)*(b_coef)
            l7_idx = (r_idx + 1)*(r_coef) + (g_idx + 1)*(g_coef) + (b_idx + 1)*(b_coef)
            # l_idx_arry = [l0_idx, l1_idx, l2_idx, l3_idx, l4_idx, l5_idx, l6_idx, l7_idx]
            
            """
            以下で `% total_index` をしているのは端点対策。
            r, g, b のいずれかが 1.0 だと端点が足りなくなる。
            なので適当なLUTを当てる。
            最終的にそのLUT値は掛け算で 0.0 になるし。
            """
            l_0 = lut[l0_idx % total_index]
            l_1 = lut[l1_idx % total_index]
            l_2 = lut[l2_idx % total_index]
            l_3 = lut[l3_idx % total_index]
            l_4 = lut[l4_idx % total_index]
            l_5 = lut[l5_idx % total_index]
            l_6 = lut[l6_idx % total_index]
            l_7 = lut[l7_idx % total_index]
            # l_array = [lut[x] for x in l_idx_arry]
             
            # 線形補間処理実行
            # -----------------------------------------
            r_out = v_0*l_7[0] + v_1*l_6[0] + v_2*l_5[0] + v_3*l_4[0]\
                + v_4*l_3[0] + v_5*l_2[0] + v_6*l_1[0] + v_7*l_0[0]
            g_out = v_0*l_7[1] + v_1*l_6[1] + v_2*l_5[1] + v_3*l_4[1]\
                + v_4*l_3[1] + v_5*l_2[1] + v_6*l_1[1] + v_7*l_0[1]
            b_out = v_0*l_7[2] + v_1*l_6[2] + v_2*l_5[2] + v_3*l_4[2]\
                + v_4*l_3[2] + v_5*l_2[2] + v_6*l_1[2] + v_7*l_0[2]
                
            # 結果を out_img に詰める
            # -----------------------------------------
            out_img[h_idx][w_idx][0] = r_out
            out_img[h_idx][w_idx][1] = g_out
            out_img[h_idx][w_idx][2] = b_out

    return out_img

img = img_open_and_normalize('../Matrix/figure/src_img.png')
img_out = exec_3dlut_on_x86(img=img, lut=lut)

# 結果の表示
# ------------------------------------
cv2.imshow('preview', img_out)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [1]:
# ここまでの処理を .py ファイルにまとめた
# ---------------------------------
import sys
import numpy as np
import cv2
import imp
sys.path.append('./code')
import pycuda_3dlut as p3
imp.reload(p3)

# 3DLUTデータの作成
# ----------------------------------------
matrix_param = np.array([[0.2126, 0.7152, 0.0722],
                         [-0.114572, -0.385428, 0.5],
                         [0.5, -0.454153, -0.045847]])
kwargs = {'mtx' : matrix_param}
lut = p3.make_3dlut_data(grid_num=5, func=p3.rgb2yuv_for_3dlut, **kwargs)

# 3DLUTの適用
# ----------------------------------------
# img = p3.img_open_and_normalize('../Matrix/figure/src_img.png')
img = p3.img_open_and_normalize('./figure/grad_img.tiff')
# img = p3.img_open_and_normalize('./figure/sat_img.tiff')
img_out = p3.exec_3dlut_on_x86(img=img, lut=lut)

# 結果の表示
# ------------------------------------
cv2.imshow('preview', img_out)
cv2.waitKey(0)
cv2.destroyAllWindows()


# CUDA で実装する！

exec_3dlut_on_x86() を CUDA で実装する。
Jupyter だとチョット書きづらいので VS Code で書きます。

In [23]:
# ここまでの処理を .py ファイルにまとめた
# ---------------------------------
import sys
import numpy as np
import cv2
import imp
sys.path.append('./code')
import pycuda_3dlut as p3
imp.reload(p3)

# 3DLUTデータの作成
# ----------------------------------------
matrix_param = np.array([[0.2126, 0.7152, 0.0722],
                         [-0.114572, -0.385428, 0.5],
                         [0.5, -0.454153, -0.045847]])
kwargs = {'mtx' : matrix_param}
lut = p3.make_3dlut_data(grid_num=17, func=p3.rgb2yuv_for_3dlut, **kwargs)

# 3DLUTの適用
# ----------------------------------------
img = p3.img_open_and_normalize('../Matrix/figure/src_img2.png')
img_x86 = p3.exec_3dlut_on_x86(img=img, lut=lut)
img_gpu = p3.exec_3dlut_on_gpu(img=img, lut=lut)

# 結果の表示
# ------------------------------------
cv2.imshow('preview', img_x86)
cv2.waitKey(0)
cv2.destroyAllWindows()

./code\pycuda_3dlut.py:88: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu
C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v8.0\bin/../include\cuda_runtime.h: warning C4819: The file contains a character that cannot be represented in the current code page (932). Save the file in Unicode format to prevent data loss
c:\program files\nvidia gpu computing toolkit\cuda\v8.0\include\cuda_runtime_api.h(1946): warning C4819: The file contains a character that cannot be represented in the current code page (932). Save the file in Unicode format to prevent data loss
c:\program files\nvidia gpu computing toolkit\cuda\v8.0\include\cuda_runtime_api.h(1946): warning C4819: The file contains a character that cannot be represented in the current code page (932). Save the file in Unicode format to prevent data loss
c:\program files\nvidia gpu computing toolkit\cuda\v8.0\include\math_functions.h: warning C4819: The file contains a character that cannot be represented in th

[[[  6.   6.  13.]
  [  8.   8.  15.]
  [  7.   7.  15.]
  ..., 
  [  9.   6.   3.]
  [ 12.  10.   6.]
  [ 14.  11.   8.]]

 [[  6.   6.  12.]
  [  8.   8.  16.]
  [  8.   8.  15.]
  ..., 
  [  9.   6.   3.]
  [ 11.   8.   5.]
  [ 13.  11.   7.]]

 [[  6.   6.  12.]
  [  8.   8.  16.]
  [  8.   8.  15.]
  ..., 
  [ 10.   7.   4.]
  [  9.   6.   4.]
  [ 12.   9.   6.]]

 ..., 
 [[ 10.  11.  11.]
  [ 13.  13.  13.]
  [ 14.  14.  15.]
  ..., 
  [ 13.  10.   9.]
  [ 12.   9.   8.]
  [ 12.   9.   8.]]

 [[ 16.  16.  16.]
  [ 16.  16.  16.]
  [ 16.  16.  16.]
  ..., 
  [ 12.   9.   8.]
  [ 11.   8.   8.]
  [ 12.   9.   8.]]

 [[ 15.  15.  15.]
  [ 15.  15.  15.]
  [ 15.  15.  15.]
  ..., 
  [ 12.   9.   8.]
  [ 12.   9.   8.]
  [ 12.   9.   8.]]]
